### Install sagemaker-experiments

In [1]:
import sys
!{sys.executable} -m pip install sagemaker-experiments

In [2]:
import sagemaker
import boto3

session = sagemaker.Session()
sm = boto3.Session().client('sagemaker')
role = sagemaker.get_execution_role()

### Split Data

In [3]:
import pandas as pd

RANDOM_STATE = 99

DATA_FILE = './aws_sagemaker/scikit-learn/classification/iris_parameterized/data/iris.csv'

# load csv in memory
data = pd.read_csv(DATA_FILE)
data.head()

,Unnamed: 0,sepal length,sepal width,petal length,petal width,class
0,0,5.1,3.5,1.4,0.2,0.0
1,1,4.9,3.0,1.4,0.2,0.0
2,2,4.7,3.2,1.3,0.2,0.0
3,3,4.6,3.1,1.5,0.2,0.0
4,4,5.0,3.6,1.4,0.2,0.0


In [4]:
# split data into test and training
train_data = data.sample(frac=0.8, random_state=RANDOM_STATE)
test_data = data.drop(train_data.index)

test_data = test_data.drop(['class'],axis=1)
test_data.head()

,Unnamed: 0,sepal length,sepal width,petal length,petal width
5,5,5.4,3.9,1.7,0.4
12,12,4.8,3.0,1.4,0.1
17,17,5.1,3.5,1.4,0.3
19,19,5.1,3.8,1.5,0.3
20,20,5.4,3.4,1.7,0.2


In [5]:
# put train data into csv
TRAIN_DATA_FILE = './aws_sagemaker/scikit-learn/classification/iris_parameterized/data/train_data.csv'
train_data.to_csv(TRAIN_DATA_FILE, index=False, header=True)

# put test data into csv
TEST_DATA_FILE = './aws_sagemaker/scikit-learn/classification/iris_parameterized/data/test_data.csv'
test_data.to_csv(TEST_DATA_FILE, index=False, header=True)

### Push data to S3 bucket

#### Define S3 Paths

In [6]:
BUCKET_NAME = 'snowflake-getting-started'
BASE_PREFIX = 'iris'

INPUT_DATA_PREFIX = BASE_PREFIX+'/data/input'
TRAIN_DATA_PREFIX = BASE_PREFIX+'/data/input/train'
TEST_DATA_PREFIX = BASE_PREFIX+'/data/input/test'

EXPERIMENTS_OUTPUT_LOC = 's3://'+BUCKET_NAME+'/'+BASE_PREFIX+'/experiments'
#DEBUGGING_OUTPUT_LOC = 's3://'+BUCKET_NAME+'/'+BASE_PREFIX+'/output/debugging'
#MODELS_OUTPUT_LOC = 's3://'+BUCKET_NAME+'/output/models'

print ('Experiment metadata would be published at -',EXPERIMENTS_OUTPUT_LOC)

Experiment metadata would be published at - s3://snowflake-getting-started/iris/experiments


#### Upload data to S3

In [7]:
print ('Uploading train data to s3')

s3_input_data_path = session.upload_data(path=DATA_FILE, 
                           bucket=BUCKET_NAME, 
                           key_prefix=INPUT_DATA_PREFIX)

print ('Input data uploaded to -', s3_input_data_path)

Uploading train data to s3
Input data uploaded to - s3://snowflake-getting-started/iris/data/input/iris.csv


### Step 1 - Setup an Experiment

### Create an Experiment

In [12]:
from smexperiments.experiment import Experiment
import time

EXPERIMENT_NAME=f"experiment-iris-classification-model-{int(time.time())}"

iris_experiment = Experiment.create(
    experiment_name=EXPERIMENT_NAME,
    description="Classification of iris flowers", 
    sagemaker_boto_client=sm)

EXP_DEBUGGING_OUTPUTS=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/debugging'
EXP_TRAINED_MODELS=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/trained_models'
EXP_SOURCE_CODE=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/code'

print ('Experiment debugging data available at -',EXP_DEBUGGING_OUTPUTS)
print ('Experiment trained moddels available at -',EXP_TRAINED_MODELS)
print ('Experiment source code available at -', EXP_SOURCE_CODE)

Experiment debugging data available at - s3://snowflake-getting-started/iris/experiments/experiment-iris-classification-model-1595847474/debugging
Experiment trained moddels available at - s3://snowflake-getting-started/iris/experiments/experiment-iris-classification-model-1595847474/trained_models
Experiment source code available at - s3://snowflake-getting-started/iris/experiments/experiment-iris-classification-model-1595847474/code


### Data Pre-Processing

#### Pre-processing tasks

    This is where we would do data pre-processing and convert intput data into some features. These features would then be uploaded to S3 and would be
    used for model training.

#### Create Tracker for Pre-Processing

    Here we are doing two things:
    1. We are creating a trial-component for 'preprocessing'
    2. We are creating a tracker to how it tracks the trial-component. This includes:-
        2.1 the artefacts generated for this trial component - for e.g. the S3 location where the artefacts would be stored
        2.2 Input data location (S3)
        2.3 Output data / Features Location (S3)
        2.4 Significant parameters from pre-processing.

In [13]:
from smexperiments.tracker import Tracker

preprocessing_trial_component_name = 'Data-Preprocessing'
preprocessing_trial_metadata_prefix = BASE_PREFIX+'/'+EXPERIMENT_NAME+'/metadata'+'/'+preprocessing_trial_component_name        

with Tracker.create(display_name=preprocessing_trial_component_name, 
                    sagemaker_boto_client=sm,
                    artifact_bucket=BUCKET_NAME,
                    artifact_prefix=preprocessing_trial_metadata_prefix) as tracker:
    tracker.log_parameters({
         #Record a single parameter value for this trial component.
        "pre_processing_param": 'pre_processing_param_value'
    })
    # Record a single input artifact for this trial component.
    # we can log the s3 uri to the dataset we just uploaded
    tracker.log_input(name="iris-dataset", media_type="s3/uri", value=s3_input_data_path)
    tracker.log_output(name="iris-dataset", media_type="s3/uri", value=s3_input_data_path)
    
# we are taking the trial component which was created earlir for preprocessing. since for different training trails
# our pre-processing trial is the same hence we just attach that preprocessing component to this trial.
preprocessing_trial_component = tracker.trial_component
    
print ('Preprocessing trial metadata would be stored at -',preprocessing_trial_metadata_prefix)    

Preprocessing trial metadata would be stored at - iris/experiment-iris-classification-model-1595847474/metadata/Data-Preprocessing


### Step 2 - Track Experiment

    We next iterate through a loop to run different trials with the hyper parameters. For each iteration we do the following:
    1. Create a Trial, since each iteration represents a trial we are doing.
    2. Add the pre-processing trial component to this trial
    3. Specify a Trial Component for the training job.

In [18]:
from sagemaker.sklearn import SKLearn
from smexperiments.trial import Trial
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig, CollectionConfig

save_interval ='1'

for i, num_max_iter in enumerate([5]):
    
    # create trial
    trial_name = f"trial-iris-classification-model-{num_max_iter}-max-iter-{int(time.time())}"
    trial_metadata_prefix = BASE_PREFIX+'/'+EXPERIMENT_NAME+'/metadata'+trial_name
    print ('Trial metadata would be stored at -',trial_metadata_prefix)
    
    iris_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=iris_experiment.experiment_name,
        sagemaker_boto_client=sm,
    )
    
    # associate the proprocessing trial component with the current trial
    iris_trial.add_trial_component(preprocessing_trial_component)
    
    executor = SKLearn(entry_point='./aws_sagemaker/scikit-learn/classification/iris_parameterized/train.py',
                      train_instance_type='ml.c4.xlarge',
                      sagemaker_session = session,
                      role = role,
                      code_location  = EXP_SOURCE_CODE,
                      hyperparameters = {
                          'max_iter':num_max_iter,
                          'class_weight':'balanced'
                      },
                      input_mode='File',
                      metric_definitions=[
                        {'Name':'test:f1-score', 'Regex':'Test F1-Score: (.*)'},
                        {'Name':'test:accuracy', 'Regex':'Test Accuracy: (.*)'}
                      ],
                      enable_sagemaker_metrics=True,
                      debugger_hook_config=DebuggerHookConfig(
                                  s3_output_path=EXP_DEBUGGING_OUTPUTS, 
                                  hook_parameters={
                                    'save_interval': '10'
                                  },
                                  # Required - See https://github.com/awslabs/sagemaker-debugger/blob/master/docs/api.md#built-in-collections for supported collections
                                  collection_configs=[ 
                                      CollectionConfig( name="default", 
                                                        parameters={
                                                            'include_regex': 'Test F1-Score: (.*)',
                                                            'include_regex': 'Test Accuracy: (.*)'
                                                        } 
                                                      ), 
                                      #CollectionConfig( name="feature_importance", parameters={ "save_interval": save_interval } ), 
                                  ]
                                ),
                      output_path = EXP_TRAINED_MODELS
                )
    
    trial_component_training_job = "training-job-iris-classification-model-{}".format(int(time.time()))
    executor.fit(
        inputs={'training': s3_input_data_path},
        job_name=trial_component_training_job,
        experiment_config={
            "TrialName": iris_trial.trial_name,
            "TrialComponentDisplayName": "Model-Training",
        },
        logs=True,
        wait=True
    )
    
    # give it a while before dispatching the next training job
    time.sleep(2)

INFO:sagemaker:Creating training-job with name: training-job-iris-classification-model-1595848541


Trial metadata would be stored at - iris/experiment-iris-classification-model-1595847474/metadatatrial-iris-classification-model-5-max-iter-1595848541
2020-07-27 11:15:41 Starting - Starting the training job...
2020-07-27 11:15:44 Starting - Launching requested ML instances.........
2020-07-27 11:17:20 Starting - Preparing the instances for training......
2020-07-27 11:18:23 Downloading - Downloading input data...
2020-07-27 11:19:10 Training - Training image download completed. Training in progress..2020-07-27 11:19:10,843 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-07-27 11:19:10,846 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-07-27 11:19:10,856 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-07-27 11:19:11,134 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-07-27 11:19:11,134 sagemaker-containers INFO     Generating 

In [16]:
training_job_desc = executor.jobs[-1].describe()
print (training_job_desc['TrainingJobName'])

client = boto3.client('logs')
client.create_export_task(
     logGroupName='/aws/sagemaker/TrainingJobs',
     destination='sagemaker-getting-started',
     destinationPrefix='iris'
    )

training-job-iris-classification-model-1595847478


ParamValidationError: Parameter validation failed:
Missing required parameter in input: "from"
Missing required parameter in input: "to"

In [ ]:
executor.debugger_hook_config

### Compare the model training runs for an experiment

Now we will use the analytics capabilities of Python SDK to query and compare the training runs for identifying the best model produced by our experiment. You can retrieve trial components by using a search expression.

### Some Simple Analyses

In [ ]:
search_expression = {
    "Filters":[
        {
            "Name": "TrialComponentName",
            "Operator": "Contains",
            "Value": "iris",
        }
    ],
}

In [ ]:
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.session import Session

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(boto3.Session(), sm), 
    experiment_name=iris_experiment.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.test:f1-score.max",
    metric_names=['test:f1-score'],
    sort_order="Descending",    
    #metric_names=['test:f1-score'],
    parameter_names=['max_iter', 'class_weight']
)
trial_component_analytics.dataframe()

In [ ]:
lineage_table = ExperimentAnalytics(
    sagemaker_session=Session(boto3.Session(), sm), 
    search_expression={
        "Filters":[{
            "Name": "Parents.TrialName",
            "Operator": "Equals",
            "Value": 'iris-training-job-20-max-iter-1595707533'
        },
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }]
    },
    sort_by="CreationTime",
    sort_order="Ascending",
)

lineage_table.dataframe()

### Evaluating Debugging Outputs

In [ ]:
!pip install smdebug

In [ ]:
import smdebug
from smdebug.trials import create_trial

trial = create_trial(EXP_DEBUGGING_OUTPUTS)

##### Plot Metrics

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import re

def get_data(trial, tname):
    """
    For the given tensor name, walks though all the iterations
    for which you have data and fetches the values.
    Returns the set of steps and the values.
    """
    tensor = trial.tensor(tname)
    steps = tensor.steps()
    vals = [tensor.value(s) for s in steps]
    return steps, vals

def plot_collection(trial, collection_name, regex='.*', figsize=(8, 6)):
    """
    Takes a `trial` and a collection name, and 
    plots all tensors that match the given regex.
    """
    fig, ax = plt.subplots(figsize=figsize)
    sns.despine()

    tensors = trial.collection(collection_name).tensor_names

    for tensor_name in sorted(tensors):
        if re.match(regex, tensor_name):
            steps, data = get_data(trial, tensor_name)
            ax.plot(steps, data, label=tensor_name)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_xlabel('Iteration')

In [ ]:
plot_collection(trial, "metrics")

##### Plot Feature Importance

In [ ]:
def plot_feature_importance(trial, importance_type="weight"):
    SUPPORTED_IMPORTANCE_TYPES = ["weight", "gain", "cover", "total_gain", "total_cover"]
    if importance_type not in SUPPORTED_IMPORTANCE_TYPES:
        raise ValueError(f"{importance_type} is not one of the supported importance types.")
    plot_collection(
        trial,
        "feature_importance",
        regex=f"feature_importance/{importance_type}/.*")
    
 plot_feature_importance(trial)    

### Step - Real Time Predictions

#### Deploy Model to an Endpoint

In [ ]:
predictor = executor.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
print('\nModel Deployed!')

print (predictor.endpoint)

#### Realtime Inference

    Lookup the predictor via the endpoint name & call predict on it

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv(DATA_FILE,engine='python')
X = data.iloc[:,1:5]
y = data.iloc[:,5]

train_x, test_x, train_y, test_y = train_test_split(X,y)
pred_y = predictor.predict(test_x)
print (pred_y)
print (test_y)